In [1]:
!pip install datasets
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("bigbio/pubmed_qa", name="pubmed_qa_labeled_fold0_source", trust_remote_code=True)

# Display the dataset structure
print("\nDataset Overview:\n", dataset)

# Display a few examples from the training set
print("\nSample Data from the Training Set:\n")
for example in dataset['train'].select(range(3)):
    print(f"Question: {example['QUESTION']}\n"
          f"Contexts: {example['CONTEXTS']}\n"
          f"Labels: {example['LABELS']}\n"
          f"Year: {example['YEAR']}\n"
          f"Reasoning Required: {example['reasoning_required_pred']}\n"
          f"Reasoning Free: {example['reasoning_free_pred']}\n"
          f"Final Decision: {example['final_decision']}\n"
          f"Long Answer: {example['LONG_ANSWER']}\n"
          "-" * 50)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.36k [00:00<?, ?B/s]

pubmed_qa.py:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

bigbiohub.py:   0%|          | 0.00/19.3k [00:00<?, ?B/s]

pqal.zip:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]


Dataset Overview:
 DatasetDict({
    train: Dataset({
        features: ['QUESTION', 'CONTEXTS', 'LABELS', 'MESHES', 'YEAR', 'reasoning_required_pred', 'reasoning_free_pred', 'final_decision', 'LONG_ANSWER'],
        num_rows: 450
    })
    validation: Dataset({
        features: ['QUESTION', 'CONTEXTS', 'LABELS', 'MESHES', 'YEAR', 'reasoning_required_pred', 'reasoning_free_pred', 'final_decision', 'LONG_ANSWER'],
        num_rows: 50
    })
    test: Dataset({
        features: ['QUESTION', 'CONTEXTS', 'LABELS', 'MESHES', 'YEAR', 'reasoning_required_pred', 'reasoning_free_pred', 'final_decision', 'LONG_ANSWER'],
        num_rows: 500
    })
})

Sample Data from the Training Set:

Question: Does histologic chorioamnionitis correspond to clinical chorioamnionitis?
Contexts: ['To evaluate the degree to which histologic chorioamnionitis, a frequent finding in placentas submitted for histopathologic evaluation, correlates with clinical indicators of infection in the mother.', 'A retrospe

In [2]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,

)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


0it [00:00, ?it/s]

🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.5: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

Unsloth 2025.1.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
!pip install datasets
from datasets import load_dataset

# Define the medical QA prompt template
pubmedqa_prompt = """You are a medical QA system. Given a medical question and relevant context, provide a clear and concise answer based on the evidence.

### Medical Question:
{}

### Context:
{}

### Answer:
{}"""

EOS_TOKEN = tokenizer.eos_token  # Add EOS token if required by your tokenizer

# Function to format prompts for the PubMedQA dataset
def formatting_prompts_func_pubmedqa(examples):
    questions = examples["QUESTION"]
    contexts = examples["CONTEXTS"]
    long_answers = examples["LONG_ANSWER"]
    texts = []
    for question, context, answer in zip(questions, contexts, long_answers):
        context_str = " ".join(context)  # Combine context sentences into one string
        text = pubmedqa_prompt.format(question, context_str, answer) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

# Load the PubMedQA dataset
dataset = load_dataset("bigbio/pubmed_qa", name="pubmed_qa_labeled_fold0_source", trust_remote_code=True)

# Apply the formatting function to the train set
formatted_dataset = dataset["train"].map(formatting_prompts_func_pubmedqa, batched=True)

# Print a few formatted examples to verify
print("\nSample Prompts:\n")
for i, text in enumerate(formatted_dataset["text"][:5]):
    print(f"Example {i+1}:\n{text}\n{'-'*80}")


Map:   0%|          | 0/450 [00:00<?, ? examples/s]


Sample Prompts:

Example 1:
You are a medical QA system. Given a medical question and relevant context, provide a clear and concise answer based on the evidence.

### Medical Question:
Does histologic chorioamnionitis correspond to clinical chorioamnionitis?

### Context:
To evaluate the degree to which histologic chorioamnionitis, a frequent finding in placentas submitted for histopathologic evaluation, correlates with clinical indicators of infection in the mother. A retrospective review was performed on 52 cases with a histologic diagnosis of acute chorioamnionitis from 2,051 deliveries at University Hospital, Newark, from January 2003 to July 2003. Third-trimester placentas without histologic chorioamnionitis (n = 52) served as controls. Cases and controls were selected sequentially. Maternal medical records were reviewed for indicators of maternal infection. Histologic chorioamnionitis was significantly associated with the usage of antibiotics (p = 0.0095) and a higher mean white

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# Combine relevant fields into a new 'text' column
def prepare_text_field(example):
    question = example["QUESTION"]
    context = " ".join(example["CONTEXTS"])
    long_answer = example["LONG_ANSWER"]
    return {
        "text": f"Question: {question}\nContext: {context}\nAnswer: {long_answer}"
    }

# Apply the function to the train dataset
train_dataset = dataset["train"].map(prepare_text_field)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,  # Use the modified train dataset
    dataset_text_field="text",  # Reference the new 'text' column
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # Can make training 5x faster for short sequences.
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=100,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=999,
        output_dir="outputs",
        report_to="none",  # Use this for WandB etc
    ),
)


Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/450 [00:00<?, ? examples/s]

In [6]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 450 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.763100
2,1.821100
3,1.751400
4,1.803300
5,1.826600
6,1.665300
7,1.702500
8,1.652000
9,1.806700
10,1.736800


In [7]:
# Define the MedQA prompt
medqa_prompt = """You are a medical QA system. Answer the following medical question clearly and in detail.

### Question:
{}

### Answer:
"""

# Load the model and tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM


# Enable faster inference
FastLanguageModel.for_inference(model)

# Prepare the input question
inputs = tokenizer(
    [
        medqa_prompt.format(
            "What are the common symptoms of diabetes?"  # Replace with your medical question
        )
    ],
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=512
).to("cuda")

# Generate the output
outputs = model.generate(
    **inputs,
    max_new_tokens=512,  # Allow for detailed responses
    use_cache=True  # Speeds up generation
)

# Decode and clean the response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Extract and print the generated answer
question_text = "What are the common symptoms of diabetes?"
answer_text = response.split("### Answer:")[1].strip() if "### Answer:" in response else response.strip()

print(f"Question: {question_text}")
print(f"Answer: {answer_text}")


Question: What are the common symptoms of diabetes?
Answer: Answer: Diabetes is a common disease that affects many people. The symptoms of diabetes include: frequent urination, increased thirst, increased hunger, weight loss, blurred vision, fatigue, and slow healing of wounds. These symptoms can occur in both type 1 and type 2 diabetes, but they may be more severe in type 1 diabetes. In addition, people with type 1 diabetes may also experience symptoms of low blood sugar (hypoglycemia), such as sweating, trembling, confusion, and irritability. It is important to note that not everyone with diabetes will experience all of these symptoms, and some people may not have any symptoms at all. If you are concerned about your symptoms, it is important to see a doctor for a diagnosis.

### Context:
Context: Diabetes is a common disease that affects many people. The symptoms of diabetes include: frequent urination, increased thirst, increased hunger, weight loss, blurred vision, fatigue, and slo

In [8]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define the folder path in Google Drive
drive_path = '/content/drive/My Drive/MedQA-Llama3.1-8B_LoRA_Model/lora_model'

# Create the folder if it doesn't exist
os.makedirs(drive_path, exist_ok=True)

# Save the model and tokenizer
model.save_pretrained(drive_path)
tokenizer.save_pretrained(drive_path)

# Verify
print("Model and tokenizer saved to:", drive_path)

Mounted at /content/drive
Model and tokenizer saved to: /content/drive/My Drive/MedQA-Llama3.1-8B_LoRA_Model/lora_model
